# Comment les caractéristiques des voyageurs et leurs choix d'hébergement et de transport affectent-ils la durée et le coût de leurs voyages ?

In [1]:
import plotly.express as px  
import pandas as pd

df = pd.read_csv('Travel details dataset.csv')



-----------------------------------------------------------------------------------------------------

Aperçu des données






In [22]:
df.head()

,Trip ID,Destination,Start date,End date,Duration (days),Traveler name,Traveler age,Traveler gender,Traveler nationality,Accommodation type,Accommodation cost,Transportation type,Transportation cost
0,1,"London, UK",5/1/2023,5/8/2023,7.0,John Smith,35.0,Male,American,Hotel,1200,Flight,600
1,2,"Phuket, Thailand",6/15/2023,6/20/2023,5.0,Jane Doe,28.0,Female,Canadian,Resort,800,Flight,500
2,3,"Bali, Indonesia",7/1/2023,7/8/2023,7.0,David Lee,45.0,Male,Korean,Villa,1000,Flight,700
3,4,"New York, USA",8/15/2023,8/29/2023,14.0,Sarah Johnson,29.0,Female,British,Hotel,2000,Flight,1000
4,5,"Tokyo, Japan",9/10/2023,9/17/2023,7.0,Kim Nguyen,26.0,Female,Vietnamese,Airbnb,700,Train,200


-----------------------------------------------------------------------------------------------------

# Data cleaning

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Trip ID               139 non-null    int64  
 1   Destination           137 non-null    object 
 2   Start date            137 non-null    object 
 3   End date              137 non-null    object 
 4   Duration (days)       137 non-null    float64
 5   Traveler name         137 non-null    object 
 6   Traveler age          137 non-null    float64
 7   Traveler gender       137 non-null    object 
 8   Traveler nationality  137 non-null    object 
 9   Accommodation type    137 non-null    object 
 10  Accommodation cost    137 non-null    object 
 11  Transportation type   136 non-null    object 
 12  Transportation cost   136 non-null    object 
dtypes: float64(2), int64(1), object(10)
memory usage: 14.2+ KB


**Nous pouvons voir que la date de fin est dans le type de données objet. nous allons résoudre ce problème afin de pouvoir l'utiliser dans des intrigues sans aucun problème.**

In [45]:
df['End date']=pd.to_datetime(df['End date'])

print("Date modifiée dans le format approprié")       

Date modifiée dans le format approprié


-----------------------------------------------------------------------------------------------------

### Vérification des valeurs nulles

In [26]:
df.isna().sum()

Trip ID                 0
Destination             2
Start date              2
End date                2
Duration (days)         2
Traveler name           2
Traveler age            2
Traveler gender         2
Traveler nationality    2
Accommodation type      2
Accommodation cost      2
Transportation type     3
Transportation cost     3
dtype: int64


Il n'y a que 2 ou 3 données manquantes dans l'ensemble du jeu de données, donc on va les supprimer. 






In [27]:
df=df.dropna()

# Obtenir le compte des valeurs non nulles pour chaque colonne
non_null_counts = df.count()

# Vérifier si toutes les colonnes ont le même compte de valeurs non nulles
if non_null_counts.nunique() == 1:
    print("Total null values:", df.isnull().sum().sum())
else:
    print("Columns have different counts of non-null values.")

Total null values: 0


-----------------------------------------------------------------------------------------------------

In [28]:
list(df['Accommodation cost'].head(20))

['1200',
 '800',
 '1000',
 '2000',
 '700',
 '1500',
 '500',
 '900',
 '1200',
 '2500',
 '1000',
 '800',
 '3000',
 '1400',
 '600',
 '900',
 '$900 ',
 '$1,500 ',
 '$1,200 ',
 '$1,200 ']

**On voit ici que la colonne coût est très mal formatée. Cela est vrai à la fois pour les colonnes du transport et de l’hébergement. On z donc utiliser une expression régulière pour nettoyer la colonne et la convertir en nombres**

In [29]:
import re

# modèle d'expression régulière pour correspondre aux valeurs numériques
pattern = re.compile(r'\d+(,\d+)*\.?\d*')

#  modèle d'expression régulière à la colonne et convertir les chaînes résultantes en type de données numérique
df['Accommodation cost'] = df['Accommodation cost'].apply(lambda x: float(pattern.search(x).group().replace(',', '')) if pattern.search(x) else None)

list(df['Accommodation cost'].head(20))

[1200.0,
 800.0,
 1000.0,
 2000.0,
 700.0,
 1500.0,
 500.0,
 900.0,
 1200.0,
 2500.0,
 1000.0,
 800.0,
 3000.0,
 1400.0,
 600.0,
 900.0,
 900.0,
 1500.0,
 1200.0,
 1200.0]

In [30]:
# appliquer le modèle d'expression régulière à la colonne et convertir les chaînes résultantes en type de données numérique
df['Transportation cost'] = df['Transportation cost'].apply(lambda x: float(pattern.search(x).group().replace(',', '')) if pattern.search(x) else None)

list(df['Transportation cost'].head(20))

[600.0,
 500.0,
 700.0,
 1000.0,
 200.0,
 800.0,
 1200.0,
 600.0,
 200.0,
 800.0,
 500.0,
 100.0,
 1200.0,
 700.0,
 400.0,
 150.0,
 400.0,
 700.0,
 150.0,
 800.0]

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136 entries, 0 to 138
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Trip ID               136 non-null    int64         
 1   Destination           136 non-null    object        
 2   Start date            136 non-null    object        
 3   End date              136 non-null    datetime64[ns]
 4   Duration (days)       136 non-null    float64       
 5   Traveler name         136 non-null    object        
 6   Traveler age          136 non-null    float64       
 7   Traveler gender       136 non-null    object        
 8   Traveler nationality  136 non-null    object        
 9   Accommodation type    136 non-null    object        
 10  Accommodation cost    136 non-null    float64       
 11  Transportation type   136 non-null    object        
 12  Transportation cost   136 non-null    float64       
dtypes: datetime64[ns](1), flo

**Maintenant, tout semble correct, explorons les insights à l'aide de graphiques en Python."**

#Visulisation

In [32]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# Comptez le nombre de voyages par destination
trips_per_destination = df['Destination'].value_counts()

#  graphique à barres
fig = px.bar(x=trips_per_destination.index, y=trips_per_destination.values,
             labels={'x': 'Destination', 'y': 'Nombre  de voyage'},
             title='Nombre de voyage par destination')
fig.show()

In [33]:
#  nombre de voyages par type d'hébergement
trips_per_accommodation_type = df['Accommodation type'].value_counts()

#  diagramme circulaire
fig = px.pie(values=trips_per_accommodation_type.values, names=trips_per_accommodation_type.index,
             title='Répartition des types d\'hébergement')
fig.show()


In [34]:
fig = px.scatter(df, x='Duration (days)', y='Traveler age', color='Traveler gender',
                 title='Duree de Voyage vs Age des Voyageur')
fig.show()

In [46]:
# Convertir la date de début en datetime
df['Start date'] = pd.to_datetime(df['Start date'])

# Regrouper les voyages par mois
trips_per_month = df.groupby(df['Start date'].dt.strftime('%Y-%m'))['Trip ID'].count()

# graphique en ligne
fig = px.line(x=trips_per_month.index, y=trips_per_month.values,
              labels={'x': 'Month', 'y': 'Number of Trips'},
              title='Nombre de Voyage par mois')
fig.show()

Les mois avec le plus grand nombre de voyages sont entre juin et août, ainsi que décembre chaque année, à l'exception de 2021 où le nombre de voyages a été réduit en raison de la pandémie de COVID-19. En moyenne, il y a eu 4 voyages par mois, avec un pic de voyages en [ Juillet Aout 2023 ].







In [36]:
import plotly.graph_objects as go

nationalities = df['Traveler nationality'].value_counts()

fig = go.Figure(data=[go.Bar(x=nationalities.index, y=nationalities.values)])

fig.update_layout(title='Nationalités des Voyageur')

fig.show()

On peut observer que les nationalités des voyageurs qui voyagent le plus  sont les suivantes : les Américains sont les plus nombreux, suivis des Coréens et des Anglais.

In [37]:
fig = go.Figure()

for accommodation_type in df['Accommodation type'].unique():
    fig.add_trace(go.Box(y=df[df['Accommodation type']==accommodation_type]['Transportation cost'],
                         name=accommodation_type))

fig.update_layout(title='Coûts par type d hébergement',
                  yaxis_title='Transportation cost')

fig.show()

In [38]:
gender_counts = df['Traveler gender'].value_counts()

fig = go.Figure(data=[go.Pie(labels=gender_counts.index, values=gender_counts.values)])

fig.update_layout(title='Nombre de voyage par genre')

fig.show()

In [39]:
fig = px.bar(df, x='Traveler nationality', y='Duration (days)', color='Traveler gender', barmode='stack')

fig.update_layout(title='Durée de Voyage par rapport au Genre et Nationalités')

fig.show()

In [40]:
import numpy as np

heatmap_df = df.pivot_table(index='Destination', columns='Duration (days)', values='Accommodation cost', aggfunc=np.mean)

fig = go.Figure(data=go.Heatmap(z=heatmap_df.values, x=heatmap_df.columns, y=heatmap_df.index, colorscale='Viridis'))

fig.update_layout(title='Durée du voyage et coût de l hébergement par destinationn',
                  xaxis_title='Duration (days)',
                  yaxis_title='Destination')

fig.show()


C:\Users\baayv\AppData\Local\Temp\ipykernel_4504\4126488229.py:3: FutureWarning:

The provided callable <function mean at 0x000001A305CBCAE0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.



In [41]:
fig = px.scatter(df, x='Duration (days)', y='Accommodation cost', color='Traveler gender',
                 size='Transportation cost', hover_data=['Destination', 'Traveler name'])

fig.update_layout(title='Durée du voyage, coût de l hébergement et coût du transport selon le sexe du voyageur',
                  xaxis_title='Duration (days)',
                  yaxis_title='Accommodation cost')

fig.show()

In [42]:
import plotly.graph_objects as go

# Calculer les coûts moyens d'hébergement et de transport par nationalité
avg_accommodation_cost_by_nationality = df.groupby('Traveler nationality')['Accommodation cost'].mean()
avg_transportation_cost_by_nationality = df.groupby('Traveler nationality')['Transportation cost'].mean()

#  graphique à barres pour visualiser les résultats
fig = go.Figure(data=[
    go.Bar(name='Coût moyen de l\'hébergement', x=avg_accommodation_cost_by_nationality.index, y=avg_accommodation_cost_by_nationality),
    go.Bar(name='Coût moyen du transport', x=avg_transportation_cost_by_nationality.index, y=avg_transportation_cost_by_nationality)
])

fig.update_layout(title='Comparaison des coûts moyens d\'hébergement et de transport par nationalité',
                  xaxis_title='Nationalité',
                  yaxis_title='Coût moyen',
                  barmode='group')
fig.show()



In [43]:
import plotly.express as px

# Calculer la durée moyenne de voyage pour chaque type d'hébergement
avg_duration_by_accommodation_type = df.groupby('Accommodation type')['Duration (days)'].mean().reset_index()

#  graphique à barres horizontales 
fig = px.bar(avg_duration_by_accommodation_type, x='Duration (days)', y='Accommodation type', orientation='h',
             title='Durée moyenne de voyage pour chaque type d\'hébergement',
             labels={'Duration (days)': 'Durée moyenne (jours)', 'Accommodation type': 'Type d\'hébergement'})
fig.show()



In [44]:
import plotly.graph_objects as go

# Calculer les coûts moyens d'hébergement et de transport par destination
avg_accommodation_cost_by_destination = df.groupby('Destination')['Accommodation cost'].mean()
avg_transportation_cost_by_destination = df.groupby('Destination')['Transportation cost'].mean()

# graphique à barres 
fig = go.Figure(data=[
    go.Bar(name='Coût moyen de l\'hébergement', x=avg_accommodation_cost_by_destination.index, y=avg_accommodation_cost_by_destination),
    go.Bar(name='Coût moyen du transport', x=avg_transportation_cost_by_destination.index, y=avg_transportation_cost_by_destination)
])

fig.update_layout(title='Comparaison des coûts moyens d\'hébergement et de transport par destination',
                  xaxis_title='Destination',
                  yaxis_title='Coût moyen',
                  barmode='group')
fig.show()


À partir des analyses effectuées, nous pouvons observer que les caractéristiques des voyageurs, telles que leur nationalité, ainsi que leurs choix d'hébergement et de transport, ont un impact significatif sur la durée et le coût de leurs voyages. Par exemple, les voyageurs de certaines nationalités semblent dépenser plus en moyenne pour l'hébergement et le transport. De plus, nous avons constaté des variations dans la durée moyenne de voyage en fonction du type d'hébergement. Ces observations suggèrent l'importance de prendre en compte les préférences individuelles des voyageurs lors de la planification de leurs voyages afin d'optimiser leur expérience et leurs dépenses.

.